<a href="https://colab.research.google.com/github/JKaly-prog/ML-Emotor/blob/first_branch/tdms_mat_to_parquet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Code to reduce the size of emotors measurement samples**

In [9]:
import sys
!{sys.executable} -m pip install nptdms

In [10]:
from nptdms import TdmsFile

### Visualizing .tdms and .mat files from Google Drive files

In [11]:
import numpy as np
import pandas as pd
from nptdms import TdmsFile
from google.cloud import storage

In [12]:
# df = pd.read_csv('/content/drive/MyDrive/Emotor_short/0Nm_BPFI_03.tdms') # This caused a UnicodeDecodeError

with TdmsFile.open('/home/jhord/code/JKaly-prog/ML-Emotor/data/0Nm_BPFI_03.tdms') as tdms_file:
    # You can inspect the TDMS file structure:
    # print(tdms_file)

    # To get a DataFrame from a specific group or channel, you need to specify it.
    # For example, to get all data as a DataFrame:
    # (Note: This might need adjustment based on your specific TDMS file structure)
    df_03 = tdms_file.as_dataframe()

    # Display the first few rows of the DataFrame
    display(df_03.head())

    # You might want to access specific groups or channels like this:
    # group = tdms_file['Measured Data']
    # channel = group['Voltage']
    # channel_data = channel.data
    # df = pd.DataFrame({'Voltage': channel_data})

    print(df_03.shape)

,/'Log'/'cDAQ9185-1F486B5Mod1/ai0',/'Log'/'cDAQ9185-1F486B5Mod1/ai1',/'Log'/'cDAQ9185-1F486B5Mod2/ai0',/'Log'/'cDAQ9185-1F486B5Mod2/ai2',/'Log'/'cDAQ9185-1F486B5Mod2/ai3'
0,27.607992,28.217591,1.894377,0.949463,-2.165271
1,27.607992,28.217591,2.128889,0.926111,-2.309938
2,27.607992,28.217591,2.373001,0.882154,-2.526938
3,27.607992,28.217591,2.087747,1.193974,-2.444867
4,27.607992,28.217591,2.393572,1.125291,-2.718899


(1536492, 5)


## Read .parquet file since bucket in GCP

In [13]:
# Verify file existence in Google Cloud Storage
from google.cloud import storage

client = storage.Client()
bucket = client.bucket("emotor-dataset-raw")
blob = bucket.blob("current_temp_short/0Nm_BPFI_03.parquet")

print(blob.exists())

True


In [1]:
import pyarrow
import pyarrow.lib as pl

print(pyarrow.__version__)
print(hasattr(pl, "PyExtensionType"))

15.0.2
True


In [15]:
# Explore Parquet File
import pandas as pd

path = "gs://emotor-dataset-raw/current_temp_short/0Nm_BPFI_10.parquet"

df = pd.read_parquet(path)

print(df.shape)
print(df.head())

(0, 0)
Empty DataFrame
Columns: []
Index: []


In [17]:
tdms_path = "/home/jhord/code/JKaly-prog/ML-Emotor/data/0Nm_BPFI_03.tdms"  # ajusta ruta real
tdms = TdmsFile.read(tdms_path)

for g in tdms.groups():
    print("\nGROUP:", g.name)
    for ch in g.channels():
        arr = ch[:]
        print(f"  - {ch.name:60s} len={len(arr)}  min={arr.min():.3f} max={arr.max():.3f}  unit={getattr(ch,'unit_string',None)}")
        # propiedades útiles
        # print("    props:", ch.properties)


GROUP: Test Information

GROUP: Log
  - cDAQ9185-1F486B5Mod1/ai0                                     len=1536492  min=27.598 max=28.527  unit=None
  - cDAQ9185-1F486B5Mod1/ai1                                     len=1536492  min=28.204 max=28.736  unit=None
  - cDAQ9185-1F486B5Mod2/ai0                                     len=1536492  min=-4.503 max=4.713  unit=None
  - cDAQ9185-1F486B5Mod2/ai2                                     len=1536492  min=-4.634 max=4.506  unit=None
  - cDAQ9185-1F486B5Mod2/ai3                                     len=1536492  min=-3.560 max=3.642  unit=None


In [19]:
import numpy as np
import pandas as pd
from nptdms import TdmsFile

tdms_path = "/home/jhord/code/JKaly-prog/ML-Emotor/data/0Nm_BPFI_03.tdms"  # ruta real
tdms = TdmsFile.read(tdms_path)

data = {}
for group in tdms.groups():
    for ch in group.channels():
        arr = np.asarray(ch[:])
        if arr.ndim > 1:
            arr = arr.reshape(-1)
        col = f"{group.name}/{ch.name}"
        data[col] = arr

df = pd.DataFrame(data)

print("DF shape:", df.shape)
print("Columns:", df.columns.tolist())

# Guardado local
out_path = "0Nm_BPFI_03.parquet"
df.to_parquet(out_path, index=False)
print("Saved:", out_path)


DF shape: (1536492, 5)
Columns: ['Log/cDAQ9185-1F486B5Mod1/ai0', 'Log/cDAQ9185-1F486B5Mod1/ai1', 'Log/cDAQ9185-1F486B5Mod2/ai0', 'Log/cDAQ9185-1F486B5Mod2/ai2', 'Log/cDAQ9185-1F486B5Mod2/ai3']
Saved: 0Nm_BPFI_03.parquet


In [20]:
import pandas as pd

path = "gs://emotor-dataset-raw/current_temp_short/0Nm_BPFI_03.parquet"
df = pd.read_parquet(path)

print(df.shape)
df.head()

(1536492, 5)


,Log/cDAQ9185-1F486B5Mod1/ai0,Log/cDAQ9185-1F486B5Mod1/ai1,Log/cDAQ9185-1F486B5Mod2/ai0,Log/cDAQ9185-1F486B5Mod2/ai2,Log/cDAQ9185-1F486B5Mod2/ai3
0,27.607992,28.217591,1.894377,0.949463,-2.165271
1,27.607992,28.217591,2.128889,0.926111,-2.309938
2,27.607992,28.217591,2.373001,0.882154,-2.526938
3,27.607992,28.217591,2.087747,1.193974,-2.444867
4,27.607992,28.217591,2.393572,1.125291,-2.718899


In [23]:
import io
import numpy as np
import pandas as pd
from tqdm import tqdm
from nptdms import TdmsFile
from google.cloud import storage

BUCKET_NAME = "emotor-dataset-raw"
INPUT_PREFIX = "current,temp/"          # OJO: tiene coma
OUTPUT_PREFIX = "current_temp_short/"   # destino

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)


def list_tdms_files(prefix: str) -> list[str]:
    """List all .tdms files under a GCS prefix."""
    blobs = client.list_blobs(BUCKET_NAME, prefix=prefix)
    return [b.name for b in blobs if b.name.lower().endswith(".tdms")]


def tdms_bytes_to_dataframe(tdms_bytes: bytes) -> pd.DataFrame:
    """Parse TDMS bytes -> DataFrame of channels (flattened to 1D)."""
    tdms = TdmsFile.read(io.BytesIO(tdms_bytes))

    data = {}
    lengths = []

    for group in tdms.groups():
        for ch in group.channels():
            arr = np.asarray(ch[:])
            if arr.ndim > 1:
                arr = arr.reshape(-1)  # a 1D

            col = f"{group.name}/{ch.name}"
            data[col] = arr
            lengths.append(len(arr))

    if not data:
        return pd.DataFrame()

    # Alinear longitudes (por seguridad): truncar todo al mínimo
    min_len = min(lengths)
    for k in list(data.keys()):
        data[k] = data[k][:min_len]

    df = pd.DataFrame(data)
    return df

In [ ]:
def convert_one_tdms_to_parquet_gcs(tdms_blob_name: str, overwrite: bool = False) -> None:
    """
    Convert a single TDMS file stored in Google Cloud Storage (GCS) into Parquet and upload it back to GCS.

    Workflow:
    1) Build output parquet path using OUTPUT_PREFIX and the TDMS base filename.
    2) Skip conversion if output exists and overwrite=False.
    3) Download TDMS bytes from GCS into memory.
    4) Parse TDMS bytes into a pandas DataFrame using `tdms_bytes_to_dataframe`.
    5) Validate that the DataFrame is not empty (guard rail).
    6) Serialize DataFrame to Parquet in-memory using a BytesIO buffer.
    7) Upload the resulting Parquet file to GCS.

    Parameters
    ----------
    tdms_blob_name : str
        Full blob name (path) to the input TDMS file in GCS.
        Example: "current,temp/0Nm_BPFI_03.tdms"
    overwrite : bool, default False
        If False, the function will skip conversion when the output parquet already exists.
        If True, the output parquet will be overwritten.

    Returns
    -------
    None

    Raises
    ------
    ValueError
        If the generated DataFrame is empty (0 rows or 0 columns). In that case, the parquet
        will NOT be uploaded.
    google.api_core.exceptions.GoogleAPIError
        If download or upload operations fail (raised by the GCS client libraries).
    """

    base_name = tdms_blob_name.split("/")[-1].replace(".tdms", ".parquet")
    out_blob_name = f"{OUTPUT_PREFIX}{base_name}"

    out_blob = bucket.blob(out_blob_name)
    if out_blob.exists() and not overwrite:
        print(f"⏭️  Ya existe, salto: {out_blob_name}")
        return

    print(f"\n📄 Procesando: gs://{BUCKET_NAME}/{tdms_blob_name}")

    # Download TDMS
    in_blob = bucket.blob(tdms_blob_name)
    tdms_bytes = in_blob.download_as_bytes()

    # Convert
    df = tdms_bytes_to_dataframe(tdms_bytes)

    # Guard rails: no subir parquets vacíos
    if df.shape[0] == 0 or df.shape[1] == 0:
        raise ValueError(f"❌ DataFrame vacío para {tdms_blob_name} (df.shape={df.shape}). No se sube parquet.")

    print(f"✅ DF: shape={df.shape} cols={df.shape[1]}")

    # Write parquet to memory
    buf = io.BytesIO()
    df.to_parquet(buf, index=False, engine="pyarrow")
    buf.seek(0)

    # Upload
    out_blob.upload_from_file(buf, content_type="application/octet-stream")
    print(f"☁️  Subido: gs://{BUCKET_NAME}/{out_blob_name}")


🔎 TDMS encontrados: 45 en 'current,temp/'


Convirtiendo TDMS → Parquet:   0%|          | 0/45 [00:00<?, ?it/s]


📄 Procesando: gs://emotor-dataset-raw/current,temp/0Nm_BPFI_03.tdms
✅ DF: shape=(1536492, 5) cols=5


Convirtiendo TDMS → Parquet:   2%|▏         | 1/45 [00:12<09:31, 13.00s/it]

☁️  Subido: gs://emotor-dataset-raw/current_temp_short/0Nm_BPFI_03.parquet

📄 Procesando: gs://emotor-dataset-raw/current,temp/0Nm_BPFI_10.tdms
✅ DF: shape=(1536492, 5) cols=5


...

☁️  Subido: gs://emotor-dataset-raw/current_temp_short/4Nm_Unbalance_1751mg.parquet

📄 Procesando: gs://emotor-dataset-raw/current,temp/4Nm_Unbalance_2239mg.tdms
✅ DF: shape=(3072983, 5) cols=5


Convirtiendo TDMS → Parquet:  98%|█████████▊| 44/45 [10:51<00:16, 16.79s/it]

☁️  Subido: gs://emotor-dataset-raw/current_temp_short/4Nm_Unbalance_2239mg.parquet

📄 Procesando: gs://emotor-dataset-raw/current,temp/4Nm_Unbalance_3318mg.tdms
✅ DF: shape=(3072983, 5) cols=5


Convirtiendo TDMS → Parquet: 100%|██████████| 45/45 [11:13<00:00, 14.98s/it]

☁️  Subido: gs://emotor-dataset-raw/current_temp_short/4Nm_Unbalance_3318mg.parquet


In [ ]:

def run_batch(overwrite: bool = False):

    """
    Batch-convert all TDMS files found under INPUT_PREFIX in GCS into Parquet and upload them to OUTPUT_PREFIX.

    This function:
    - Lists TDMS blobs under INPUT_PREFIX via `list_tdms_files`.
    - Iterates through each file with a progress bar (tqdm).
    - Converts each TDMS to Parquet using `convert_one_tdms_to_parquet_gcs`.
    - Catches exceptions per-file so the batch continues even if one file fails.

    Parameters
    ----------
    overwrite : bool, default False
        If False, skip files whose parquet outputs already exist.
        If True, overwrite existing parquet outputs.

    Returns
    -------
    None
    """

    tdms_files = list_tdms_files(INPUT_PREFIX)
    print(f"🔎 TDMS encontrados: {len(tdms_files)} en '{INPUT_PREFIX}'")

    for f in tqdm(tdms_files, desc="Convirtiendo TDMS → Parquet"):
        try:
            convert_one_tdms_to_parquet_gcs(f, overwrite=overwrite)
        except Exception as e:
            print(f"\n⚠️ Error con {f}: {e}")


In [ ]:

if __name__ == "__main__":

    """
    Entry point for running the TDMS→Parquet batch conversion as a script.

    Executes `run_batch(overwrite=False)` by default to avoid overwriting outputs.
    """

    run_batch(overwrite=False)

In [ ]:
from nptdms import TdmsFile
import numpy as np

tdms = TdmsFile.read("0Nm_BPFO_03.tdms")

for g in tdms.groups():
    for ch in g.channels():
        arr = np.asarray(ch[:])
        print(g.name, ch.name, arr.shape)

In [39]:
BUCKET_NAME = "emotor-dataset-raw"
BLOB_NAME = "current,temp/0Nm_BPFO_03.tdms"

# Cliente GCP
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
blob = bucket.blob(BLOB_NAME)

# Descargar TDMS a memoria
tdms_bytes = blob.download_as_bytes()

# Leer TDMS desde RAM
tdms = TdmsFile(io.BytesIO(tdms_bytes))

In [42]:
for g in tdms.groups():
    for ch in g.channels():
        arr = ch[:]
        print(g.name, ch.name, arr.dtype)

Log cDAQ9185-1F486B5Mod1/ai0 float64
Log cDAQ9185-1F486B5Mod1/ai1 float64
Log cDAQ9185-1F486B5Mod2/ai0 float64
Log cDAQ9185-1F486B5Mod2/ai2 |V8
Log cDAQ9185-1F486B5Mod2/ai3 |V8


In [43]:
BUCKET_NAME = "emotor-dataset-raw"
PREFIX = "current,temp/"

FILES = [
    "0Nm_BPFO_03.tdms",
    "0Nm_BPFO_10.tdms",
    "0Nm_BPFO_30.tdms",
    "2Nm_BPFO_03.tdms",
    "2Nm_BPFO_10.tdms",
    "2Nm_BPFO_30.tdms",
    "4Nm_BPFO_03.tdms",
    "4Nm_BPFO_10.tdms",
    "4Nm_BPFO_30.tdms",
]

# Cliente GCP
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)


# Reporte de canales con dtype V8
v8_report = []

for file_name in FILES:
    blob_path = f"{PREFIX}{file_name}"
    print(f"\n📂 Procesando: {blob_path}")

    blob = bucket.blob(blob_path)
    tdms_bytes = blob.download_as_bytes()
    tdms = TdmsFile(io.BytesIO(tdms_bytes))

    for group in tdms.groups():
        for channel in group.channels():
            arr = channel[:]

            if arr.dtype == "|V8":
                v8_report.append({
                    "file": file_name,
                    "group": group.name,
                    "channel": channel.name,
                    "dtype": str(arr.dtype)
                })

                print(f"⚠️ V8 detectado → {group.name} / {channel.name}")

if not v8_report:
    print("✅ No se encontraron canales con dtype |V8")
else:
    for r in v8_report:
        print(
            f"{r['file']} | "
            f"{r['group']} | "
            f"{r['channel']} | "
            f"dtype={r['dtype']}"
        )


📂 Procesando: current,temp/0Nm_BPFO_03.tdms
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai2
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai3

📂 Procesando: current,temp/0Nm_BPFO_10.tdms
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai2
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai3

📂 Procesando: current,temp/0Nm_BPFO_30.tdms
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai2
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai3

📂 Procesando: current,temp/2Nm_BPFO_03.tdms
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai2
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai3

📂 Procesando: current,temp/2Nm_BPFO_10.tdms
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai2
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai3

📂 Procesando: current,temp/2Nm_BPFO_30.tdms
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai2
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai3

📂 Procesando: current,temp/4Nm_BPFO_03.tdms
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai2
⚠️ V8 detectado → Log / cDAQ9185-1F486B5Mod2/ai3

In [46]:
import io
import numpy as np
import pandas as pd
from google.cloud import storage
from nptdms import TdmsFile

BUCKET_NAME = "emotor-dataset-raw"
BLOB_NAME   = "current,temp/0Nm_BPFO_03.tdms"

client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
blob = bucket.blob(BLOB_NAME)

tdms_bytes = blob.download_as_bytes()
tdms = TdmsFile(io.BytesIO(tdms_bytes))

data = {}
v8_channels = []
lengths = {}

for g in tdms.groups():
    for ch in g.channels():
        arr = ch[:]
        col = f"{g.name}/{ch.name}"
        lengths[col] = len(arr)

        if arr.dtype == "|V8":
            v8_channels.append(col)
            # Series con el mismo largo del canal (índice 0..n-1)
            data[col] = pd.Series(np.nan, index=np.arange(len(arr)), dtype="float64")
        else:
            # Convertimos a numérico seguro y lo guardamos como Series
            s = pd.to_numeric(pd.Series(arr), errors="coerce")
            data[col] = s.reset_index(drop=True)

# ✅ Esto YA NO falla con largos distintos: alinea por índice y rellena NaN
df = pd.DataFrame(data)

print("✅ DataFrame creado:", df.shape)
print("📏 Longitudes (min/max):", min(lengths.values()), "/", max(lengths.values()))
print(f"⚠️ Canales reemplazados (|V8 → NaN): {len(v8_channels)}")
for c in v8_channels[:20]:
    print(" -", c)
if len(v8_channels) > 20:
    print(f" ... ({len(v8_channels)-20} más)")

df.head()


✅ DataFrame creado: (1536492, 5)
📏 Longitudes (min/max): 0 / 1536492
⚠️ Canales reemplazados (|V8 → NaN): 2
 - Log/cDAQ9185-1F486B5Mod2/ai2
 - Log/cDAQ9185-1F486B5Mod2/ai3


,Log/cDAQ9185-1F486B5Mod1/ai0,Log/cDAQ9185-1F486B5Mod1/ai1,Log/cDAQ9185-1F486B5Mod2/ai0,Log/cDAQ9185-1F486B5Mod2/ai2,Log/cDAQ9185-1F486B5Mod2/ai3
0,28.993472,29.995407,-1.185821,NaN,NaN
1,28.993472,29.995407,-1.137821,NaN,NaN
2,28.993472,29.995407,-0.945823,NaN,NaN
3,28.993472,29.995407,-0.951309,NaN,NaN
4,28.993472,29.995407,-0.930738,NaN,NaN
